### Import Libraries and Modules for Analysis

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
pd.set_option("display.max_rows", 300)

### Function to Scrape Data from Top Cities

In [13]:
def get_descriptions(max_results):
    
    max_results_per_city = max_results
    
    city_set = ["New+York",
                "Chicago",
                "San+Francisco",
                "Austin",
                "Seattle",
                "Los+Angeles", 
                "Philadelphia",
                "Atlanta",
                "Dallas",
                "Pittsburgh",
                "Portland",
                "Phoenix",
                "Denver",
                "Houston",
                "Miami",
                "Washington+DC",
                "Boulder"]

    job_list = []

    for city in city_set:
        for start in range(0, max_results_per_city, 10):
            page = requests.get("http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=" 
                                + str(city) + "&start=" + str(start))
            time.sleep(3)  #ensuring at least 3 seconds between page grabs
            soup = BeautifulSoup(page.text, 
                            "lxml", 
                             from_encoding="utf-8")
        for div in soup.find_all(name="div", attrs={"class":"row"}): 
            #specifying row num for index of job posting in dataframe
            #creating an empty list to hold the data for each posting
            job_post = [] 
            #append city name
            job_post.append(city) 
            #grabbing job title
            for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
                job_post.append(a["title"]) 
            #grabbing company name
            if div.find_all(name="span", attrs={"class":"company"}):
                company = div.find_all(name="span", attrs={"class":"company"}) 
                if len(company) > 0: 
                    for b in company:
                        job_post.append(b.text.strip()) 
                else: 
                    if div.find_all(name="span", attrs={"class":"result-link-source"}):
                        sec_try = div.find_all(name="span", 
                                               attrs={"class":"result-link-source"})
                        for span in sec_try:
                            job_post.append(span.text) 
                    else:
                        job_post.append("Nothing_found")
            else:
                job_post.append("Nothing_found")           
        #grabbing location name
            if div.findAll("span", attrs={"class": "location"}):
                c = div.findAll("span", attrs={"class": "location"})
                for span in c: 
                    job_post.append(span.text) 
            else:
                job_post.append("Nothing_found")
        #grabbing summary text
            if div.findAll("span", attrs={"class": "summary"}):
                d = div.findAll("span", attrs={"class": "summary"}) 
                for span in d:
                    job_post.append(span.text.strip()) 
            else:
                job_post.append("Nothing_found") 
        #grabbing salary
            try:
                job_post.append(div.find('nobr').text) 
            except:
                    try:
                        div_two = div.find(name="div", attrs={"class":"sjcl"}) 
                        div_three = div_two.find("div") 
                        job_post.append(div_three.text.strip())
                    except:
                        job_post.append("Nothing_found") 
        #grabbing full details
            e = div.findAll(attrs={"class":"jobtitle turnstileLink"},
                            href=True)
            for i in e:
                page2 = requests.get('https://www.indeed.com'+i['href'])
                soup2 = BeautifulSoup(page2.text, 
                        "lxml", 
                    from_encoding="utf-8")
                for j in soup2.findAll(attrs={"class":
                                              "jobsearch-JobComponent-description icl-u-xs-mt--md"}):
                    job_post.append(j.text)

        #appending list of job post info to dataframe at index num
            job_list.append(job_post)
    return job_list

### Run function and Scrape Data

In [14]:
job_data = get_descriptions(max_results=300)

In [ ]:
### Create 

In [15]:
indeed_df = pd.DataFrame(job_data,
                         columns=['City',
                                  'Title',
                                  'Company',
                                  'Location',
                                  'Summary',
                                  'Salary',
                                  'Description'])

In [19]:
indeed_df[~indeed_df['Description'].isnull()].head()

,City,Title,Company,Location,Summary,Salary,Description
11,New+York,Data Scientist I,AIG,Nothing_found,Data Scientists collaborate with AIG’s Life an...,"AIG\n\n\n3,108 reviews",Functional Area:\nSM - Sales & Marketing\nEsti...
12,New+York,Data Scientist,Aetna,Nothing_found,Demonstrates proficiency in several areas of d...,"Aetna\n\n\n4,010 reviews",POSITION SUMMARY\nUses statistical predictive ...
13,New+York,Data Scientist/Data Science Instructor,NYC Data Science Academy,Nothing_found,Instructors have the opportunity of taking par...,NYC Data Science Academy\n\n\n13 reviews,"Part-time, Temporary, ContractAbout UsJust as ..."
14,New+York,Data Scientist,Ascensia Diabetes Care,Nothing_found,Experience in and understanding of data and in...,Ascensia Diabetes Care\n\n\n32 reviews,Ascensia Diabetes Care is a global specialist ...
15,New+York,"Staff Data Scientist - Technology - New York, NY",Finra,Nothing_found,Collaborate with Data Engineers to specify Dat...,Finra\n\n\n88 reviews,We Work to Protect Investors. Join our Team.\n...


In [188]:
indeed_df[~indeed_df['Description'].isnull()]
         .to_csv('~/Documents/CUNY/Data/Data-607-Project-3/data/indeed_jobs_revised.csv')